# Training Module

In [1]:
import os
os.chdir('genre_classification_289a/src')

In [2]:
import torch
import torch.nn as nn
import os
from model import STN, MLP
import torch.optim as optim

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [4]:
from features import get_data_loaders, FramedFeatureDataset, FeatureDataset, DatasetSettings

### DCNN Training

In [5]:
#STN targets
agfs = [] #'subgenre', 'mfcc'
genre = True #False if not genre STN

target = None

if (genre and len(agfs) > 0) or (len(agfs) > 1):
    raise 'Error: can only train one STN target at a time'
elif genre:
    target = 'genre'
elif len(agfs) == 1:
    target = agfs[0]
else:
    raise 'Error: could not parse training target'

print('Training STN target: {}'.format(target))
    
#dataset
dataset_name = 'fma_small'

Training STN target: genre


In [6]:
settings = DatasetSettings(dataset_name, 'fma_metadata')
dataset = FramedFeatureDataset(settings,  agfs=agfs, genre=genre)
print("Num genres: ", settings.num_genres)
print(settings.genre_counts)

Num genres:  8
Pop              1000
International    1000
Instrumental     1000
Folk             1000
Rock              999
Experimental      999
Electronic        999
Hip-Hop           997
Name: genre_top, dtype: int64


In [7]:
output_size = settings.num_genres
stn = STN(output_size)
stn.to(device)
stn = nn.DataParallel(stn)

In [8]:
## Training Parameters
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(stn.parameters(), lr=0.001)
epochs = 30
batch_size = 64
valid_split = 0.2

In [9]:
trainloader, validloader = get_data_loaders(dataset, batch_size, valid_split)

In [10]:
import torch.nn.functional as F
from sklearn.metrics import f1_score

def validate(stn, label_name):
    
    with torch.no_grad():
        stn.eval()
        
        all_pred = []
        all_true = []
        
        for i, data in enumerate(validloader, 0):
            inputs, labels = data[0].to(device), data[1][label_name].to(device)
            
            out = stn(inputs)
            loss = F.cross_entropy(out, labels)
            
            all_pred.append(out.argmax(dim=1))
            all_true.append(labels)
            
        all_pred = torch.cat(all_pred)
        all_true = torch.cat(all_true)
        
        curr_f1 = f1_score(all_true.cpu(), all_pred.cpu(), average='micro')
        
        print('Validation f1 score: {}'.format(curr_f1))

In [11]:
#Train it
for epoch in range(epochs):  # loop over the dataset multiple times
    
    print('Starting epoch', epoch + 1)
    
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data[0].to(device), data[1]['genre'].to(device)  #data[1]['{argument for agf being trained}']
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = stn(inputs)
        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        

        # print statistics
        running_loss += loss.item()
        if i % 50 == 49:    # print every 30 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 30))
            running_loss = 0.0
            
    if epoch % 5 == 4:
        validate(stn, 'genre')
        stn.train()

print('Finished Training')

Starting epoch 1
[1,    50] loss: 3.098
[1,   100] loss: 2.852
[1,   150] loss: 2.601
[1,   200] loss: 2.545
[1,   250] loss: 2.441
[1,   300] loss: 2.404
[1,   350] loss: 2.395
[1,   400] loss: 2.340
[1,   450] loss: 2.300
[1,   500] loss: 2.342
[1,   550] loss: 2.224
[1,   600] loss: 2.121
[1,   650] loss: 2.161
[1,   700] loss: 2.206
Starting epoch 2
[2,    50] loss: 2.155
[2,   100] loss: 2.099
[2,   150] loss: 2.118
[2,   200] loss: 2.057
[2,   250] loss: 2.095
[2,   300] loss: 2.055
[2,   350] loss: 2.120
[2,   400] loss: 2.054
[2,   450] loss: 2.031
[2,   500] loss: 2.020
[2,   550] loss: 2.000
[2,   600] loss: 1.963
[2,   650] loss: 1.968
[2,   700] loss: 2.008
Starting epoch 3
[3,    50] loss: 1.917
[3,   100] loss: 1.909
[3,   150] loss: 1.899
[3,   200] loss: 1.858
[3,   250] loss: 1.927
[3,   300] loss: 1.862
[3,   350] loss: 1.995
[3,   400] loss: 1.864
[3,   450] loss: 1.887
[3,   500] loss: 1.803
[3,   550] loss: 1.853
[3,   600] loss: 1.859
[3,   650] loss: 1.830
[3,   

### Save Model

In [12]:
model_file = '../models/DCNN_{}_{}'.format(dataset_name, target)
torch.save(stn, model_file)

## Load & Eval Model

In [11]:
model_file = '../models/DCNN_{}_{}'.format(dataset_name, target)
model = torch.load(model_file)
model

DataParallel(
  (module): STN(
    (layer1): Sequential(
      (0): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
      (1): ELU(alpha=1.0)
      (2): MaxPool2d(kernel_size=(2, 1), stride=(2, 1), padding=0, dilation=1, ceil_mode=False)
    )
    (layer2): Sequential(
      (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ELU(alpha=1.0)
      (3): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
      (4): Dropout(p=0.1, inplace=False)
    )
    (layer3): Sequential(
      (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ELU(alpha=1.0)
      (2): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    )
    (layer4): Sequential(
      (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, mome

In [12]:
validate(model, target)

Validation f1 score: 0.9080511125011168


## Previous STN Val Accuracies
FMA_SMALL
* genre: 89.X% (can't reload model to validate b/c didn't set a random seed during training and the random split of training / validation data is different so we're mixing training data into the validation set and getting  unrealistically high validation accuracies)
* mfcc: (prev. incorrect val: 97.36%)

FMA_MED
* genre: 89.3%
